In [12]:
from pyteomics import fasta
import pandas as pd
import os
import random
import bisect
from collections import defaultdict, Counter

In [13]:
# Replace path to your input fasta file
infastafile = '/home/mark/test_protein_inference/SP_human_12112020.fasta'

sp_human = dict()
with fasta.read(infastafile) as f:
    for name, sequence in f:
        sp_human[name.split('|')[1]] = sequence

In [14]:
def cdf(weights):
    total = sum(weights)
    result = []
    cumsum = 0
    for w in weights:
        cumsum += w
        result.append(cumsum / total)
    return result

def choice(population, weights):
    assert len(population) == len(weights)
    cdf_vals = cdf(weights)
    x = random.random()
    idx = bisect.bisect(cdf_vals, x)
    return population[idx]

In [15]:
ff = "".join(sp_human.values())
weights = (pd.Series(Counter(ff)) / len(ff)).values
population = (pd.Series(Counter(ff)) / len(ff)).index

In [16]:
weights

array([2.13210226e-02, 7.01296578e-02, 8.32970596e-02, 5.73202700e-02,
       4.76880370e-02, 5.35183507e-02, 7.10154522e-02, 5.96488575e-02,
       2.62231223e-02, 5.63628376e-02, 4.33753228e-02, 3.58980389e-02,
       6.57433694e-02, 4.73719145e-02, 3.64938489e-02, 2.66371971e-02,
       6.31583283e-02, 9.96277991e-02, 1.21557569e-02, 2.30105885e-02,
       3.16826608e-06])

In [17]:
population

Index(['M', 'A', 'S', 'K', 'Q', 'T', 'E', 'V', 'H', 'R', 'I', 'N', 'G', 'D',
       'F', 'Y', 'P', 'L', 'W', 'C', 'U'],
      dtype='object')

In [18]:
# Fraction of proteins in the database which should be used for FAKE protein generation. 1.0 means 100%:
per = 1.0

smpl = random.sample(list(sp_human.keys()), int(per * len(sp_human)))
len(smpl)

20385

In [19]:
fasta_1 = sp_human.copy()
percents = [0.01, 0.05, 0.25]
for start, p  in zip(range(len(percents)), percents):
    for name in smpl[start::3]:
        try:
            positions = random.sample(range(len(fasta_1[name])), max(1, int(p * len(fasta_1[name]))))
        except ValueError:
            print(number, name, fasta_1[name])
            positions = range(number)
        seq = fasta_1[name]
        for i in positions:
            cur_aa = seq[i]
            new_aa = seq[i]
            while cur_aa == new_aa:
                new_aa = choice(population, weights)
            seq = seq[:i] + new_aa + seq[i + 1:]
        fasta_1["ZFAKE{}_".format(start + 1) + name] = seq

In [20]:
# Replace path to your output fasta file with FAKE proteins
path_to_output_fasta = '/home/mark/test_protein_inference/swiss_prot_human_and_3_group_of_fakes.fasta'

# Write output fasta file with both target and FAKE proteins
fasta.write(fasta_1.items(), 
            output=path_to_output_fasta, 
            file_mode="w")

# Write output fasta file with both target and FAKE proteins and reversed DECOYS for all target and FAKE proteins
fasta.write_decoy_db(source=path_to_output_fasta,
                    output=open(path_to_output_fasta.replace('.fasta', '_with_reversed_decoys.fasta'), 'w'), mode='reverse').close()

# Write output fasta file with both target and FAKE proteins and shuffled DECOYS for all target and FAKE proteins
fasta.write_decoy_db(source=path_to_output_fasta,
                    output=open(path_to_output_fasta.replace('.fasta', '_with_shuffled_decoys.fasta'), 'w'), mode='shuffle').close()